### CROP YIELD PREDICTION USING LINEAR REGRESSION

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('./dataset.csv')
data

,id,water,uv,area,fertilizer_usage,yield,pesticides,region,categories
0,169,5.615,65.281,3.230,0,7.977,8.969,0,"b,a,c"
1,476,7.044,73.319,9.081,0,23.009,7.197,0,"c,a,d"
2,152,5.607,60.038,2.864,2,23.019,7.424,0,"d,a"
3,293,9.346,64.719,2.797,2,28.066,1.256,0,d
4,10,7.969,NaN,5.407,1,29.140,0.274,0,"c,d"
...,...,...,...,...,...,...,...,...,...
995,614,6.379,94.132,11.571,5,98.673,2.891,6,"c,b"
996,490,6.980,82.727,11.264,5,101.090,3.500,6,"d,c,b"
997,392,NaN,58.218,11.859,4,103.673,4.005,6,"c,b,a"
998,773,10.070,67.905,11.567,4,104.343,2.339,6,a


In [11]:
data.describe()

,id,water,uv,area,fertilizer_usage,yield,pesticides,region
count,1000.000000,958.000000,949.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,499.500000,12.223546,73.957488,8.098848,2.12300,58.758571,3.452301,3.039000
std,288.819436,172.335566,9.904063,2.692632,1.52256,24.563683,2.076921,1.883886
min,0.000000,0.072000,45.264000,0.263000,0.00000,2.843000,0.014000,0.000000
25%,249.750000,4.584750,66.502000,6.297000,1.00000,40.698000,1.804500,2.000000
50%,499.500000,6.476000,73.689000,7.987500,2.00000,55.602500,3.275500,2.000000
75%,749.250000,8.758750,80.554000,9.900250,3.00000,73.645500,4.916000,5.000000
max,999.000000,5340.000000,106.310000,18.311000,5.00000,148.845000,9.532000,6.000000


In [8]:
data.shape

(1000, 9)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1000 non-null   int64  
 1   water             958 non-null    float64
 2   uv                949 non-null    float64
 3   area              1000 non-null   float64
 4   fertilizer_usage  1000 non-null   int64  
 5   yield             1000 non-null   float64
 6   pesticides        1000 non-null   float64
 7   region            1000 non-null   int64  
 8   categories        1000 non-null   object 
dtypes: float64(5), int64(3), object(1)
memory usage: 70.4+ KB
